СОЗДАНИЕ КОПИИ ТАБЛИЦЫ

На протяжении всего модуля мы будем производить множество тренировочных преобразований с нашей таблицей. Поэтому, чтобы не переопределять переменную melb_data и тем самым не повредить первоначальный DataFrame, создадим копию melb_df с помощью метода copy():


In [ ]:
melb_df = melb_data.copy()
melb_df.head()


 
На протяжении всего модуля наши данные будут множество раз меняться (будут добавляться и удаляться столбцы), поэтому, чтобы у вас не было расхождений с ответами в заданиях и в уроке, советуем следить за изменениями на протяжении модуля и фиксировать их у себя в ноутбуке.

⭐ *Лайфхак*. Старайтесь всегда оставлять переменную с первоначальным DataFrame неизменной, создавайте копию исходной таблицы и совершайте преобразования на ней. Это оградит вас от ошибок, которые можно совершить при подготовке данных. Например, если вы понимаете, что преобразование оказалось неудачным, достаточно будет лишь запустить ячейку, в которой вы производите копирование, а не читать таблицу заново. Особенно критичным это может быть, когда количество строк в таблице исчисляется миллионами и её чтение занимает до нескольких минут.



УДАЛЕНИЕ СТОЛБЦОВ

→ Среди списка базовых операций над столбцами в Pandas важное место занимает возможность удаления столбцов из таблицы. Это может быть полезно, например, когда в данных есть признаки, которые не несут полезной информации.

Представим, что мы хотим построить модель, которая бы предсказывала цену объекта недвижимости в Мельбурне. Даже не будучи профессиональными риелторами, мы можем легко сделать следующие выводы:

-	цена объекта никак не зависит от его порядкового номера (столбец index);
-	признак, описывающий долготу и широту в виде кортежа Coordinates, дублирует информацию, представленную в столбцах Longitude и Lattitude.

За удаление строк и столбцов в таблице отвечает метод **drop()**.

Кликните на плашку, чтобы увидеть информацию ↓

**Основные параметры метода drop()**
•	labels — порядковые номера или имена столбцов, которые подлежат удалению; если их несколько, то передаётся список;
•	axis — ось совершения операции, axis=0 — удаляются строки, axis=1 — удаляются столбцы;
•	inplace — если параметр выставлен на True, происходит замена изначального DataFrame на новый, при этом метод ничего не возвращает; если на False — возвращается копия DataFrame, из которой удалены указанные строки (столбцы), при этом первоначальный DataFrame не изменяется; по умолчанию параметр равен False.

Удалим столбцы index и Coordinates из таблицы с помощью метода drop(). Выведем первые пять строк таблицы и убедимся, что всё прошло успешно.



In [ ]:
melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
melb_df.head()


Альтернативный вариант:

In [ ]:

melb_df.drop(['index','Coordinates'],axis=1,inplace=True)
melb_df.head()


 
МАТЕМАТИЧЕСКИЕ ОПЕРАЦИИ СО СТОЛБЦАМИ

→ Pandas поддерживает базовые математические операции между столбцами: столбцы можно складывать, вычитать, умножать, делить между собой, а также возводить в степень. С помощью таких операций мы можем создавать новые признаки или производить преобразования над старыми.

Причём все операции со столбцами совершаются поэлементно, очень быстро, а самое главное — без написания циклов.
Такая производительность достигается за счёт того, что все математические операции со столбцами выполняются на языке программирования С, что значительно повышает скорость вычислений по сравнению с перебором элементов в цикле. 

Например, давайте создадим переменную total_rooms, в которой будем хранить общее количество комнат в здании. Для этого выполним сложение столбцов с количеством комнат, ванн и спален:



In [ ]:
total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
display(total_rooms)


 
А теперь введём признак MeanRoomsArea, который соответствует средней площади одной комнаты для каждого объекта. Для этого разделим площадь здания на полученное ранее общее количество комнат:


In [ ]:
melb_df['MeanRoomsArea'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsArea'])


 
Можно ввести ещё один интересный признак — AreaRatio, коэффициент соотношения площади здания (BuildingArea) и площади участка (Landsize). Для этого разницу двух площадей поделим на их сумму:


In [ ]:
diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])


 
Что показывает такой коэффициент? Если присмотреться, можно увидеть, что AreaRatio лежит в интервале от -1 до 1.
Рассмотрим три случая, чтобы понять его значение:

-	Если рассматриваемые площади равны, то числитель дроби зануляется и коэффициент тоже равен 0.

-	Если одна из площадей начинает доминировать над другой, то коэффициент начинает расти в отрицательную сторону, если площадь участка больше площади здания, и в положительную сторону, если наоборот.

-	Наконец, в предельном случае, если площадь здания равна 0, то числитель дроби равен знаменателю со знаком минус, а коэффициент равен -1, а если площадь участка равна 0, то числитель дроби равен знаменателю со знаком плюс, а коэффициент равен 1.

Таким образом, значение в столбце AreaRatio служит своеобразным указателем соотношения площадей объекта недвижимости. Для пустырей — участков без строений — он будет равен -1, для домов без территории — 1, во всех остальных случаях мы можем увидеть, какая площадь больше — здания или участка.

✍ Мы рассмотрели базовые операции со столбцами.  Даже сейчас, не имея представления о специальных математических преобразованиях данных, мы можем создавать интересные признаки и извлекать большее количество информации из таблицы.

Подумайте, какие ещё признаки можно ввести в нашу таблицу с помощью простых математических операций.
Предлагаем вам решить несколько задач, чтобы закрепить пройденный материал ↓


In [1]:
import numpy as np

In [2]:
import pandas as pd

*Задание 2.1*

Среди приведённых ниже вариантов кода выберите тот, который найдёт квадрат цены объекта недвижимости за наименьшее время и не выдаст ошибку.
Результат должен быть занесён в переменную price_square и представлять собой объект Series:

In [4]:
melb_data = pd.read_csv('data/melb_data_ps.csv')
melb_df = melb_data.copy()
price_square = melb_df['Price'] **2
print(price_square)

0        2.190400e+12
1        1.071225e+12
2        2.146225e+12
3        7.225000e+11
4        2.560000e+12
             ...     
13575    1.550025e+12
13576    1.062961e+12
13577    1.368900e+12
13578    6.250000e+12
13579    1.651225e+12
Name: Price, Length: 13580, dtype: float64


*Задание 2.2*

Задан DataFrame customer_df, содержащий столбцы:

cust_id — идентификатор клиента;

cust_age — возраст клиента (точкой отсчёта возраста считается 2021 год);

cust_sale — персональная скидка клиента;

cust_year_birth — год рождения клиента;

cust_order — сумма заказа клиента.

In [ ]:
customer_df = pd.DataFrame({
        'number': [0, 1, 2, 3, 4],
        'cust_id': [128, 1201, 9832, 4392, 7472],
        'cust_age': [13, 21, 19, 21, 60],
        'cust_sale': [0, 0, 0.2, 0.15, 0.3],
        'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
        'cust_order': [1400, 14142, 900, 1240, 8430]
    })

Какие столбцы не несут полезной информации/дублируют информацию из других столбцов и поэтому могут быть удалены?

В качестве ответа запишите названия этих столбцов по порядку их следования в коде, через запятую, без пробелов и кавычек.

**(number,cust_age или number,cust_year_birth)**

*Задание 2.3*

Напишите функцию delete_columns(df, col=[]), которая удаляет столбцы из DataFrame и возвращает новую таблицу. Если одного из указанных столбцов не существует в таблице, то функция должна возвращать None.

Пример DataFrame:


customer_df = pd.DataFrame({
            'number': [0, 1, 2, 3, 4],
            'cust_id': [128, 1201, 9832, 4392, 7472],
            'cust_age': [13, 21, 19, 21, 60],
            'cust_sale': [0, 0, 0.2, 0.15, 0.3],
            'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
            'cust_order': [1400, 14142, 900, 1240, 8430]
        })
Примечание. Не забудьте импортировать библиотеки.

In [5]:
import numpy as np

In [6]:
import pandas as pd

In [8]:
customer_df = pd.DataFrame({
            'number': [0, 1, 2, 3, 4],
            'cust_id': [128, 1201, 9832, 4392, 7472],
            'cust_age': [13, 21, 19, 21, 60],
            'cust_sale': [0, 0, 0.2, 0.15, 0.3],
            'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
            'cust_order': [1400, 14142, 900, 1240, 8430]
        })
def delete_columns(df, col=[]):
    for cc in col:
        if cc not in df.columns:
            return None
    return df.drop(col, axis=1)
print(delete_columns)

<function delete_columns at 0x000002A4DABC8720>


*Задание 2.4*

Задан DataFrame countries_df, содержащий следующие столбцы: название страны, население (population) в миллионах человек и площадь страны (area) в квадратных километрах.

countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'area': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})

Для каждой страны рассчитайте плотность населения (количество человек на квадратный километр).

Затем по полученным данным рассчитайте среднее по плотностям населения в указанных странах. Ответ округлите до сотых.

Плотность населения рассчитывается как количество человек, проживающих на территории отдельной страны, делённое на площадь этой страны. Обратите внимание, что население в таблице представлено в миллионах.

(84.93)

In [3]:
countries_df = pd.DataFrame({
    'country': ['Англия', 'Канада', 'США', 'Россия', 'Украина', 'Беларусь', 'Казахстан'],
    'population': [56.29, 38.05, 322.28, 146.24, 45.5, 9.5, 17.04],
    'square': [133396, 9984670, 9826630, 17125191, 603628, 207600, 2724902]
})
countries_df['density'] = countries_df['population'] / countries_df['square'] * 1e6
print(round(countries_df['density'].mean(), 2))

84.93
